## Capstone Project - Columbia DSI - DiDi 
#### Authors:  Danyang Han,  Anita Pinto,  Elizabeth Yum
This file is used to convert the trip longituide/lattitude data from the trip data file to taxi zones for both pick up and drop 
Reference: https://github.com/skyetim/didi-vehicle-repositioning-strategy(Fall 2020 Capstone Project) 

In [1]:
#!pip install fiona
#!pip install shapely
#!pip install geopandas
import fiona
import pandas as pd
import shapely
import geopandas
from shapely.geometry import shape,mapping, Point, Polygon, MultiPolygon
import matplotlib.pyplot as plt


In [2]:
#df = pd.read_csv("./Data/trip_data_1.csv", error_bad_lines=False)
df_org= pd.read_csv("../data/trip_data_1.csv", error_bad_lines=False)



/Users/anita/opt/anaconda3/envs/DiDi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Fetch records for only driver 2010000001

In [3]:
dr_1 = df_org.loc[(df_org[" hack_license"]==2010000001), :]
dr_1.head(10)

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2010000001,2010000001,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:34:00,1,34.0,14.05,-73.948418,40.724590,-73.926140,40.864761
16351,2010000001,2010000001,VTS,1,NaN,2010-01-01 00:38:00,2010-01-01 00:55:00,3,17.0,4.81,-73.924988,40.861774,-73.925735,40.809376
30295,2010000001,2010000001,VTS,1,NaN,2010-01-01 01:03:00,2010-01-01 01:13:00,2,10.0,2.30,-73.937386,40.797436,-73.968666,40.794395
36468,2010000001,2010000001,VTS,1,NaN,2010-01-01 01:14:00,2010-01-01 01:18:00,2,4.0,0.76,-73.968666,40.794495,-73.961273,40.796570
308897,2010000001,2010000001,VTS,1,NaN,2010-01-01 18:42:00,2010-01-01 19:10:00,1,28.0,17.74,-73.783813,40.648659,-73.971901,40.693054
335196,2010000001,2010000001,VTS,1,NaN,2010-01-01 20:04:00,2010-01-01 20:48:00,1,44.0,18.20,-73.788696,40.641308,-73.985748,40.759399
347365,2010000001,2010000001,VTS,1,NaN,2010-01-01 20:49:00,2010-01-01 20:59:00,2,10.0,2.52,-73.986237,40.759514,-73.990334,40.736927
391885,2010000001,2010000001,VTS,1,NaN,2010-01-01 23:24:00,2010-01-01 23:32:00,1,8.0,3.73,-73.776588,40.644917,-73.806030,40.665817
649110,2010000001,2010000001,VTS,1,NaN,2010-01-02 18:00:00,2010-01-02 18:53:00,2,53.0,21.27,-73.776703,40.645378,-73.978172,40.776268
675167,2010000001,2010000001,VTS,1,NaN,2010-01-02 18:57:00,2010-01-02 19:07:00,2,10.0,1.76,-73.981781,40.773548,-73.967918,40.755630


In [4]:
#Shape of the dr_1 dataframe
dr_1.shape

(258, 14)

In [5]:
#Copy dr_1 into dff 
dff = dr_1.copy()
#df = df_org.iloc[0:1000000,:]
#print(df.shape)
#dff = df.iloc[0:1000000,:]
print(dff.shape)

(258, 14)


In [6]:
#Just checking the shapes files by importing using geopandas
shape_df = geopandas.read_file('../taxi_zones/taxi_zones.shp')
shape_df.shape

(263, 7)

In [9]:
# https://towardsdatascience.com/geospatial-operations-at-scale-with-dask-and-geopandas-4d92d00eb7e8
def assign_taxi_zones(df, lon_var, lat_var, locid_var):

    import geopandas
    from shapely.geometry import Point

    localdf = df[[lon_var, lat_var]].copy()
    
    localdf[lon_var] = localdf[lon_var].fillna(value=0.)
    localdf[lat_var] = localdf[lat_var].fillna(value=0.)
    
    shape_df = geopandas.read_file('../taxi_zones/taxi_zones.shp')
    
    shape_df = shape_df.to_crs({'init': 'epsg:4326'})
    try:
        local_gdf = geopandas.GeoDataFrame(
            localdf, crs={'init': 'epsg:4326'},
            geometry=[Point(xy) for xy in
                      zip(localdf[lon_var], localdf[lat_var])])

        local_gdf = geopandas.sjoin(
            local_gdf, shape_df, how='left', op='within')

        return local_gdf.OBJECTID.rename(locid_var)
    except ValueError as ve:
        print(ve)
        print(ve.stacktrace())
        series = localdf[lon_var]
        series = np.nan
        return series

In [10]:
dff = dff.join(assign_taxi_zones(dff, " pickup_longitude", " pickup_latitude", "pickup_taxizone_id"))
dff = dff.join(assign_taxi_zones(dff, " dropoff_longitude", " dropoff_latitude", "dropoff_taxizone_id"))

/Users/anita/opt/anaconda3/envs/DiDi/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/anita/opt/anaconda3/envs/DiDi/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
#dff.to_csv('result.csv',index=False)

In [11]:
dff.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_taxizone_id,dropoff_taxizone_id
0,2010000001,2010000001,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:34:00,1,34.0,14.05,-73.948418,40.724590,-73.926140,40.864761,112,127
16351,2010000001,2010000001,VTS,1,NaN,2010-01-01 00:38:00,2010-01-01 00:55:00,3,17.0,4.81,-73.924988,40.861774,-73.925735,40.809376,243,168
30295,2010000001,2010000001,VTS,1,NaN,2010-01-01 01:03:00,2010-01-01 01:13:00,2,10.0,2.30,-73.937386,40.797436,-73.968666,40.794395,74,151
36468,2010000001,2010000001,VTS,1,NaN,2010-01-01 01:14:00,2010-01-01 01:18:00,2,4.0,0.76,-73.968666,40.794495,-73.961273,40.796570,151,151
308897,2010000001,2010000001,VTS,1,NaN,2010-01-01 18:42:00,2010-01-01 19:10:00,1,28.0,17.74,-73.783813,40.648659,-73.971901,40.693054,132,97


In [12]:
dff.columns

Index(['medallion', ' hack_license', ' vendor_id', ' rate_code',
       ' store_and_fwd_flag', ' pickup_datetime', ' dropoff_datetime',
       ' passenger_count', ' trip_time_in_secs', ' trip_distance',
       ' pickup_longitude', ' pickup_latitude', ' dropoff_longitude',
       ' dropoff_latitude', 'pickup_taxizone_id', 'dropoff_taxizone_id'],
      dtype='object')

In [13]:
dff.to_csv("../data/dr_1_2010_01.csv")


In [15]:
df_list = []
df_list.append(dff)

In [17]:
df_org= pd.read_csv("../data/trip_data_2.csv", error_bad_lines=False)
dr_1 = df_org.loc[(df_org[" hack_license"]==2010000001), :]
dff = dr_1.copy()
dff = dff.join(assign_taxi_zones(dff, " pickup_longitude", " pickup_latitude", "pickup_taxizone_id"))
dff = dff.join(assign_taxi_zones(dff, " dropoff_longitude", " dropoff_latitude", "dropoff_taxizone_id"))
dff.to_csv("../data/dr_1_2010_02.csv")
df_list.append(dff)

/Users/anita/opt/anaconda3/envs/DiDi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/anita/opt/anaconda3/envs/DiDi/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/anita/opt/anaconda3/envs/DiDi/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-orde

In [19]:
df_org= pd.read_csv("../data/trip_data_3.csv", error_bad_lines=False)
dr_1 = df_org.loc[(df_org[" hack_license"]==2010000001), :]
dff = dr_1.copy()
dff = dff.join(assign_taxi_zones(dff, " pickup_longitude", " pickup_latitude", "pickup_taxizone_id"))
dff = dff.join(assign_taxi_zones(dff, " dropoff_longitude", " dropoff_latitude", "dropoff_taxizone_id"))
dff.to_csv("../data/dr_1_2010_03.csv")
df_list.append(dff)

/Users/anita/opt/anaconda3/envs/DiDi/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/anita/opt/anaconda3/envs/DiDi/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [20]:
dr_1_2010_Q1 = pd.concat(df_list)
dr_1_2010_Q1.shape

(773, 16)

In [ ]:
dr_1_2010_Q1.to_csv()